In [7]:
%matplotlib notebook
#from selenium import webdriver
import requests
import pandas as pd
from StringIO import StringIO
import csv
import os
from dateutil import parser

FDIR=os.getenv('FDIR')
assert FDIR is not None

def downloadSymbols(symbols):

    fromday="01"
    frommonth="00"
    fromyear="2006"
    today="24"
    tomonth="03"
    toyear="2016"

    for symbol in symbols:
        r=requests.get("http://real-chart.finance.yahoo.com/table.csv?s={}&a={}&b={}&c={}&d={}&e={}&f={}&g=d&ignore=.csv".format(symbol, frommonth, fromday, fromyear, tomonth, today, toyear))
        if r.ok:
            f=open(FDIR+'/data/{}_{}_{}_{}_{}_{}_{}.csv'.format(symbol, frommonth, fromday, fromyear, tomonth, today, toyear),'w')
            f.write(r.content)
            f.close()
            
def loadSymbols(reqsymbols):
    rootdir=FDIR+"/data"
    symbolFiles=os.listdir(rootdir)

    symbols={a.split('_')[0]:{"file":a, "data":[]} for a in symbolFiles if a.split('_')[0] in reqsymbols}
    print "Found {} files, loading symbols: {}".format(len(symbolFiles), ", ".join(symbols.keys()))

    print "Loading all symbols..."
    for symbol, pkg in symbols.items():
        fname=pkg['file']
        pkg['data']=pd.read_csv("/".join([rootdir,fname]), index_col=False, parse_dates=[0], infer_datetime_format=True)
    return symbols

In [8]:
rs={"FRESX":{'alloc':.2,'min':2500, 'sharesOwned':0}, 
    "FEMKX":{'alloc':.05,'min':2500, 'sharesOwned':0}, 
    "FBGRX":{'alloc':.3,'min':2500, 'sharesOwned':0}, 
    "FIGFX":{'alloc':.15,'min':2500, 'sharesOwned':0}}
downloadSymbols(rs.keys())

In [7]:
rs=["FFFGX"]
rs={"FRESX":{'alloc':.2,'min':2500, 'sharesOwned':0}, 
    "FEMKX":{'alloc':.05,'min':2500, 'sharesOwned':0}, 
    "FBGRX":{'alloc':.3,'min':2500, 'sharesOwned':0}, 
    "FIGFX":{'alloc':.15,'min':2500, 'sharesOwned':0}}

#TODO: need to assert we have data for each of these symbols or can run simulation

## Any two of the following three can be specified but not all three
## for now we will assume purchasePerMonth and monthsToBalance are the two specified
## purchasePerMonth is the number of purchases that can be made each month
## monthsToBalance is the number of months by which we want to have reached balanced allocation
purchasePerMonth=3; maxPurchasePerMonth=None; monthsToBalance=3
assert maxPurchasePerMonth is None and purchasePerMonth is not None and monthsToBalance is not None

## totalAtBalance is the smallest 100% total investment at which each asset class can meet its allocation %
totalAtBalance=max([(1.0/v['alloc'])*v['min'] for s,v in rs.items()])

## remianingToBalance is the amount this asset class needs to still purchase in order to reach its allocation %
for s,v in rs.items():
    v['remianingToBalance']=totalAtBalance*(1.0-v['alloc'])

## amountAtEachPurchaseToBalance is the amount of this equity that needs to be purchased at each interval to reach
## the balanced allocation in the set amount of time
for s,v in rs.items():
    v['amountAtEachPurchaseToBalance']=totalAtBalance*(1.0-v['alloc'])

# for each year
# purchase equities to balance within set amount of time
# purchase fixed percentage based on allocation after balanced
#    maintain running total of shares owned in each equity:
#        dollar amount divided by price pershare rounded to nearest share, remaining money should remain in account
#        

## Need to include bond payouts into assets
#downloadSymbols(rs.keys())
symbols=loadSymbols(rs.keys())
for symbol,pkg in symbols.items():
    print "{}: {}".format(symbol, len(pkg['data']))


Found 5 files, loading symbols: FBGRX, FIGFX, FEMKX, FRESX
Loading all symbols...
FBGRX: 2583
FIGFX: 2121
FEMKX: 2583
FRESX: 2583
